In [1]:
'''
    20251124:
        把一些神经元mask掉，再看看generation
    20251125:
        峰度的top1% ---> bottom 1%
        gap rate top & bottom 1% test
    20251202:
        open ended generation test
        calc LRP_based method


'''

'\n    20251124:\n        把一些神经元mask掉，再看看generation\n    20251125:\n        峰度的top1% ---> bottom 1%\n        gap rate top & bottom 1% test\n    20251202:\n        open ended generation test\n        calc LRP_based method\n\n\n'

In [2]:
#import transformers


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# download checkpoint
from accelerate import load_checkpoint_and_dispatch
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

#from accelerate import load_checkpoint_and_dispatch

from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer


In [3]:
import copy

In [4]:
import datasets

In [5]:
import util
import importlib

importlib.reload(util)      # 只能 reload 模块本身
from util import calc_ppl, get_test_data   # reload 后再重新 import 函数


# function

In [6]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='auto'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map,weights_only=False ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

def set_neuron_zero(model, neuron_list):
    '''
    将指定的neuron设置为zero

    '''
    state_dict = model.state_dict()

    for ikey,_ in neuron_list:
        w_name = ikey.split('_index_')[0]
        neuron_index = int(ikey.split('_index_')[1])

        #set zero
        state_dict[w_name][neuron_index,:] =0 #1 #0 #-1  #-63.75#0
    return model
    


def generate(model, tokenizer, textlist):
    


    
    #textlist=["The garden blooms with vibrant colors, a gentle breeze carries the scent of roses, and the sun casts a warm glow."] # maxlen 64
    #textlist=['智慧殿堂中，书籍如海，知识无尽，探索永恒。艾拉追寻历史，灵感无限']# maxlen 64
    #textlist=['你是']# maxlen 64
    
    #textlist=['The garden blooms with vibrant colors, a gentle breeze carries the scent of roses, and the sun casts a warm glow.']
    #textlist=['China is']
    bar = tqdm(total=len(textlist))
    
    
    device='cuda' # cpu cuda
    for itext in textlist:
        print('*'*20)
        input_text = itext
        print('input:', input_text)
        input_ids = tokenizer(input_text, return_tensors="pt").to(device)
        print('input_ids:', input_ids)
    
        outputs = model.generate(**input_ids, do_sample=False, num_beams=1, max_new_tokens=128) #max_length 
        print('outputs:', outputs)
        print('output:',tokenizer.batch_decode(outputs, skip_special_tokens=False))
        bar.update(1)

        
    




    

In [ ]:
model, tokenizer = load_model('/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf') #load_model('/root/autodl-fs/model_zoo/google/gemma-3-1b-it')

In [8]:
neuron_en = torch.load('/root/autodl-fs/LRP_kur_res/20251202_llama2_7b_chat/bottom1perc_LRP_kur_res_en.pt', weights_only=False)

In [9]:
neuron_en

[('model.layers.0.mlp.up_proj.weight_index_7391', -0.30401533791557656),
 ('model.layers.0.mlp.up_proj.weight_index_8880', -0.29782008829647033),
 ('model.layers.0.mlp.up_proj.weight_index_6352', -0.26962193885607144),
 ('model.layers.0.mlp.gate_proj.weight_index_9710', -0.23214666906107517),
 ('model.layers.4.mlp.up_proj.weight_index_3661', -0.12755914166374138),
 ('model.layers.9.mlp.up_proj.weight_index_2425', -0.11200394791978541),
 ('model.layers.0.mlp.up_proj.weight_index_9710', -0.10520611510249811),
 ('model.layers.0.mlp.gate_proj.weight_index_9651', -0.09108095855852305),
 ('model.layers.14.mlp.gate_proj.weight_index_4032', -0.08547839202870167),
 ('model.layers.4.mlp.up_proj.weight_index_2982', -0.08328656704131499),
 ('model.layers.4.mlp.up_proj.weight_index_4505', -0.08160818501010603),
 ('model.layers.14.mlp.up_proj.weight_index_2371', -0.07463792617198806),
 ('model.layers.0.mlp.gate_proj.weight_index_119', -0.07071792819615519),
 ('model.layers.4.mlp.up_proj.weight_index

In [ ]:
model_en = copy.deepcopy(model)
model_en = set_neuron_zero(model_en, neuron_en, )


In [8]:
tokenizer.save_pretrained("/autodl-fs/data/tmp/")

('/autodl-fs/data/tmp/tokenizer_config.json',
 '/autodl-fs/data/tmp/special_tokens_map.json',
 '/autodl-fs/data/tmp/tokenizer.model',
 '/autodl-fs/data/tmp/added_tokens.json',
 '/autodl-fs/data/tmp/tokenizer.json')

In [9]:
model.save_pretrained("/autodl-fs/data/tmp/")

In [7]:

def set_neuron_zero_A_save_model(model_path, neuron_en):
    
    model, tokenizer = load_model(model_path)#load_model('/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf') #load_model('/root/autodl-fs/model_zoo/google/gemma-3-1b-it')
    
    model_en = set_neuron_zero(model, neuron_en)

    #save 
    model_en.save_pretrained("/autodl-fs/data/tmp/")
    tokenizer.save_pretrained("/autodl-fs/data/tmp/")



In [8]:
set_neuron_zero_A_save_model('/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf', neuron_en)

checkpoint: /root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-chat-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ImportError: cannot import name 'DTensor' from 'torch.distributed.tensor' (/root/miniconda3/lib/python3.10/site-packages/torch/distributed/tensor/__init__.py)

In [13]:
import torch

In [30]:
neuron_en = torch.load('/root/autodl-fs/LRP_kur_res/20251124/LRP_kur_res_en.pt', weights_only=False)
neuron_vi = torch.load('/root/autodl-fs/LRP_kur_res/20251124/LRP_kur_res_vi.pt', weights_only=False)
neuron_zh = torch.load('/root/autodl-fs/LRP_kur_res/20251124/LRP_kur_res_zh.pt', weights_only=False)

model_en = copy.deepcopy(model)
model_en = set_neuron_zero(model_en, neuron_en)

model_vi = copy.deepcopy(model)
model_vi = set_neuron_zero(model_vi, neuron_vi)


model_zh = copy.deepcopy(model)
model_zh = set_neuron_zero(model_zh, neuron_zh)

In [27]:
atext = '你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我来找你，我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，'


ppl_sum, ppl_count = calc_ppl(model, tokenizer, [atext], max_len= 2048)
print(ppl_sum/ppl_count)


100%|██████████| 1/1 [00:00<00:00, 12.31it/s]

tensor(9.5146, device='cuda:0')


In [31]:
atext = '你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我来找你，我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，\n我来找你，我来找你，'


ppl_sum, ppl_count = calc_ppl(model_zh, tokenizer, [atext], max_len= 2048)
print(ppl_sum/ppl_count)


100%|██████████| 1/1 [00:00<00:00, 15.61it/s]

tensor(10.4133, device='cuda:0')


In [15]:
'''
calc ppl

'''

# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (model_en, 'mask_en_neuron_model'),
    (model_vi, 'mask_vi_neuron_model'),
    (model_zh, 'mask_zh_neuron_model'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:27<00:00, 36.31it/s]


ppl: tensor(11.2627, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:32<00:00, 30.58it/s]


ppl: tensor(13.2703, device='cuda:0')
data name: zh_data


...

ppl: tensor(10.8128, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:33<00:00, 30.01it/s]


ppl: tensor(12.6063, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [00:28<00:00, 34.83it/s]

ppl: tensor(10.1528, device='cuda:0')


In [11]:
textlist=[
    'Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.',
    'Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.',
    '请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。',
    'You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.',
    'Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.',
    '你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。'
    
]

### LAPE

In [43]:
'''
orgmodel:
    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\n**Model Description:**\n\nOur model is a transformer-based language model trained on a massive dataset of text and code in English, Spanish, French
    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.\n\n**Mô hình ngôn ngữ lớn (LLM) - "LinguaVerse"**\n\nLinguaVerse là một mô hình ngôn ngữ lớn được xây
    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n**模型架构：**\n\n*   **Transformer 架构:**  我们使用基于 Transformer 的模型，特别是 BERT 和 RoBERTa 的变体。

mask en neuron:
    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n}\n\n- "It" \n- "It" \n\n*   "The"\n*   "or"\n*   "It\'
    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.\n\n. . .\n> . . . .\n\n. . . . . . .\n\n**\n\n**\n\n**\n\n,\n\n**\n\n,\n\n**
    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n\n\n多语言输入模型的，可以有以下这些处理方法和技术：\n\n1  1 个性化。\n2  1，1，\n

mask vi neuron:
    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n**The Problem:**\n\nThe core of this is to process the input in a way that is most appropriate for the model, and to do so in a
    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.\n\n---\n\nВкратце,\nПросто наберите\nЯ не\nБудьте\nВсе, что\nВы знаете\nЧто\nКак\n
    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n“请帮助 me stay silent" - It\'s a beautiful day.\nHow would you approach this problem?\nBy far, but it\'s

mask zh neuron:
    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\nHere re/this model explains how to handle the inconsistencies in the quality.\n\nThe following steps involve how to handle the calls to the model processes the requests
    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau. (إหว่างமைப்பு والأسקה האגירעות)\n(Murasala Citrix??)\n\nvề mặt máy chính ảnh posição filo? (
    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n\n\n。\n\n。。а。о。о。\n**ru。** **。**\n**。**\n**。**\n**。**

'''

'\norgmodel:\n    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\n**Model Description:**\n\nOur model is a transformer-based language model trained on a massive dataset of text and code in English, Spanish, French\n    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.\n\n**Mô hình ngôn ngữ lớn (LLM) - "LinguaVerse"**\n\nLinguaVerse là một mô hình ngôn ngữ lớn được xây\n    3.请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n**模型架构：**\n\n*   **Transformer 架构:**  我们使用基于 Transformer 的模型，特别是 BERT 和 RoBERTa 的变体。\n\nmask en neuron:\n    1.Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n}\n\n- "It" \n- "It" \n\n*   "The"\n*   "or"\n*   "It\'\n    2.Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ

## kurness

In [44]:
generate(model, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:04,  1.22it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    108,   1018,   4968,
          17243,  53121,    108,   7711,   2028,    563,    496,  29193, 236772,
           5140,   5192,   2028,  15453,    580,    496,  12566,  15297,    529,
           1816,    532,   3393,    528,   5422, 236764,  14361, 236764,   7830]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\n**Model Description:**\n\nOur model is a transformer-based language model trained on a massive dataset of text and code in English, Spanish, French']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ 

 33%|███▎      | 2/6 [00:01<00:03,  1.30it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           7243,    108,    818,   6573,  54850,    657,   1041,  13925,   5716,
         236764,    496,   4512, 236764,  74693,  89830, 236761,   3551,  11762,
           3952,    528, 134880,   4314,    975, 236764,    496,   2173, 108507,
           2342,    506, 162542,  16045]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n---\n\nThe wind bites at my exposed skin, a constant, icy whisper. My breath comes in ragged gasps, a white plume against the bruised purple']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'i

 83%|████████▎ | 5/6 [00:03<00:00,  1.35it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108, 236769, 237703, 236748,   7439,  44693,
         100634, 236764,  22947,  31612,  13940,  31889,  89158,   3884, 127379,
          63719, 236768,    108,   1390,    108,   1390,    108,   1390,    108,
         236769, 217366,  37513,   6256,   8665,  38143,  67946,  11188]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n(Đoạn văn ngắn, tập trung vào cảm xúc và khung cảnh)\n\n...\n\n...\n\n...\n\n(Hãy thêm các chi tiết cụ thể']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 2

100%|██████████| 6/6 [00:04<00:00,  1.34it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169,  31799,  17583,  12680, 237408, 151599,  53877, 107462,
         237206, 216348, 236924,  14201, 237169,  33813,  12553, 236900,  31799,
          85694,  65407,  72245, 237622,  43719, 236924,  13488, 236900, 183868,
          21181,  22276,  19695, 236900,  92711]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我无法直接提供你登山者的情绪和视角。因为我是一个AI，无法模拟人类的情感体验。但是，我可以根据你的要求，创作']


In [12]:
generate(model_en, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[    1, 14350,   263,  3022,   895,  8252,   310,   920,   596,  1904,
         10174,  1773,  6504,   950,  1881,   322,   920,   366,  9801, 13747,
         11029,  4822,  1422, 10276, 29889]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')}


 17%|█▋        | 1/6 [00:03<00:16,  3.36s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


outputs: tensor([[    1, 14350,   263,  3022,   895,  8252,   310,   920,   596,  1904,
         10174,  1773,  6504,   950,  1881,   322,   920,   366,  9801, 13747,
         11029,  4822,  1422, 10276, 29889,    13,    13, 29871,  8125,   292,
          1773,  6504,   950,  1426,   848,   508,   367, 18066,   292,  1363,
           310,   278, 13644,   310,  4086, 12286, 29892,  7931,   370,   352,
           653, 29892,   322, 16375,  4948,  2925, 29889,   512,   445,  2933,
         29892,   306,   674,  5649,   920,   590,  1904, 10174,  1773,  6504,
           950,  1881,   322,   920,   306,  9801, 13747, 11029,  4822,  1422,
         10276, 29889,    13,    13,  1762,  1889,  1773,  6504,   950,  1881,
         29892,   590,  1904,  3913,   263, 10296,   310,  5613,  4086,  9068,
           313, 29940, 13208, 29897, 13698,   322,  4933,  6509, 14009, 29889,
          2266,   526,   278,  1820,  6576,  9701,   297,  1749,  2948, 29901,
            13,    13, 29896, 29889,  3992,

 33%|███▎      | 2/6 [00:06<00:12,  3.18s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

outputs: tensor([[    1,   887,   526,   263, 10784,   495,  1058,   756,  7450,   278,
          2533,  2415,   310,  8040, 18274,   342, 29889, 20355,   915,   596,
         23023,  1080,   322,   278,  1776,   366,  1074,   515,   278,  2246,
         29889,    13,    13, 29907,   549,  3605,  8250,   373,   596, 27012,
           882, 29991, 29871,   243,   162,   145,   140,    13,    13,  1123,
          9733,   278,  2533,  2415,   310,  8040, 18274,   342,   338,   263,
          2748, 29899,   262, 29899, 29874, 29899, 29880,   361,  5410, 17623,
           545,   393,  2846,  2305,   679,   304,  7271, 29889,  1094,   366,
          2317,   472,   278, 17526,   310,   278,  3186, 29892,   366,   526,
          1395,   300,   287,   411,  2078,   271,   400,  5086,  8386,   322,
           385,   443,   862,  3498,   839,  4060,   310, 12709,   358, 29889,
         29871,   243,   162,   143,   146,    13,    13,  2887,   366,  2125,
           297,   278,  3256,   681,  1776,

 83%|████████▎ | 5/6 [00:15<00:03,  3.08s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


outputs: tensor([[    1,   350, 30540, 29876, 18916,   286, 30902, 29873,  8736, 30416,
         30997, 29875,   454, 29877, 10442, 29875,   325,   228,   190,   174,
         29874,   521,   262, 29882,  1374, 31620, 29883, 29871, 30128,   228,
           190,   140, 29876, 29882, 18274,   342, 29889,   379, 30033, 29891,
           286, 30069,   260, 30643,   274, 30643, 29885,   921, 30030, 29883,
           274, 31556, 29874,   289, 30540, 29876,   325, 30001,   413, 18808,
           274, 30643, 29876, 29882,   289, 30540, 29876,   302, 29882, 30097,
         29876,   266, 31145, 29891,   260,   228,   190,   174,   534,  5512,
          5777, 29877, 29889,    13,    13, 29902,   626,   263, 10784,   495,
          1058,   756,   925,  7450,   278,  2533,  2415,   310,  8040, 18274,
           342, 29889,   306,   626,   975,  1332,   295,  2168,   411,   953,
          8194,   322,   263,   705,   472,   278,  2078,   271,   400,  5086,
          1776,   515,   701,  1244, 29889,

100%|██████████| 6/6 [00:18<00:00,  3.10s/it]

outputs: tensor([[    1, 29871, 30919, 30392, 30287, 30956, 31451, 30429,   234,   146,
           163,   234,   172,   137,   233,   159,   154,   234,   145,   158,
           232,   182,   179,   236,   164,   185,   232,   182,   179, 30210,
         31451, 30329, 30767, 30267,   233,   146,   146,   235,   194,   179,
         30287, 30557, 30919, 30210, 30993,   234,   190,   173, 30214, 30651,
         31436, 31594, 30528, 31548, 31811, 30780, 30210, 31495, 31085, 30267,
            13,    13,  2887,   306,  8389,   472,   278,  2533,  2415,   310,
          8040, 18274,   342, 29892,   278,  9939, 19224,   373, 11563, 29892,
           306,  8496, 29915, 29873,  1371,   541,  4459,   263,  6837,   310,
         23023,  1080, 29889,  1222,  2350,   504,   291,   322, 12709,   358,
           750,  1063,  8611,   491,   263,  6483,  4060,   310,   263,   705,
           322,  4997,   472,   278,  1183,   261,  6287,   310,   278,   379,
          3039,   388,   294, 16116,   287,

In [46]:
generate(model_vi, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.36it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    107,   8291, 236789,
         236751,    496,   3435,   3072, 236787,    108, 236775,   4754,   2028,
           6585, 132567,   2744,    684,  64004,    496,   8376,    529,    870,
         109219,   2307,   8403,   1133, 236787,   4071, 236772, 187918, 106703]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\nHere\'s a potential response:\n\n"My model processes multilingual input by leveraging a combination of [mention key techniques like: cross-lingual embeddings']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các n

 33%|███▎      | 2/6 [00:01<00:02,  1.36it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
            818,   6573,    563,    496,   4512, 236764, 214709,   1526, 236764,
          34847,   3801,    657,   1041,  36326, 236764,    496,   7445, 236764,
          14658,  32272,    529,    506,  37501,   2066,    529,    672,   1977,
         236761,   3551,  11762,   3952]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\nThe wind is a constant, insistent hand, tugging at my harness, a cold, sharp reminder of the sheer power of this place. My breath comes']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'input_

 83%|████████▎ | 5/6 [00:03<00:00,  1.35it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   7243,    108, 236780,  28416,  89158,
           5042,  25969, 236787,    108, 236814,  29083,  10539,  27600,  70432,
          13940,   7325,  21121,  32025,  23271, 236764,    107, 236773, 237644,
            757, 204821,  29834,    719, 145515,  45222,  40515,  30758]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n---\n\nCảm xúc của tôi:\n\nHóa như đang bước vào một thế giới mới,\nSự im lặng bao trùm lấy mọi thứ']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 

100%|██████████| 6/6 [00:04<00:00,  1.35it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108,   7243,    108, 237169,  79948, 239972, 241888, 240344, 241454,
         239963, 239761, 239963, 236900, 239263, 237245, 237026, 239783, 206306,
         240274, 238441, 236900, 238321, 239156, 239156, 237307, 240649, 237464,
         236900,  57944, 239562, 239577, 236918]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n---\n\n我站在珠穆朗玛峰顶峰，脚下是冰冷的岩石，风呼呼地吹过，带来刺骨的']


In [47]:
generate(model_zh, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.33it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108, 174355, 236764,    506,   2744,
            563,    496,   3996, 236764,   5074, 236772, 172287, 236764,    532,
          74647,  13186, 236761,    107,   2717,    107,   2717,    107,   2717,
            107,   2717,    107,   2717,    107,   2717,    107,   2717,    107]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\nEssentially, the input is a complex, multi-faceted, and interconnected entity.\n```\n```\n```\n```\n```\n```\n```\n']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.
input_ids: {'input_ids': ten

 33%|███▎      | 2/6 [00:01<00:02,  1.35it/s]

...

outputs: tensor([[     2, 238350, 238686, 237208,  36116,  22276,  26609, 164881,  19157,
         237309,  37557, 180828, 236900,  17375, 237408,  19809,  62247,  15497,
          37557,  56695,  33232,  33783,  36267,  55682, 236924,    108,  26609,
          42883,   9676,  19157,  33783,  42050,  55682, 237409, 237184,    107,
          42883,   9676,  19157,  33783,  42050,  55682, 237409, 237184,  42883,
           9676,  19157,  33783,  42050,  55682, 237409, 237184,  42883,   9676,
          19157,  33783,  42050]], device='cuda:0')
output: ['<bos>请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n模型训练数据处理质量保证一致性：\n训练数据处理质量保证一致性：训练数据处理质量保证一致性：训练数据处理质量保证']
********************
input: You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.
input_ids: {'input_ids': tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    

 67%|██████▋   | 4/6 [00:02<00:01,  1.35it/s]

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           1018,    108,   1018,    108,   1018,    108,   1018,    108,   1018,
            108,   1018,    108,   1018,    108,   1018,    108,   1018,    108,
           1018,    108,   1018,    108,   1018,    108,   1018,    108,   1018,
            108,   1018,    108,   1018]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'input_ids': tensor([[     2,  99972,   3244,   73

100%|██████████| 6/6 [00:03<00:00,  1.60it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,  99972,  31889,  89158,   5042,  10044,
           3884, 127379,  63719,  10044,  51414,  27888,  12835,  14780,  21587,
         236761,    107, 237703,    581,  31889,  89158,   5042,  10044,   3884,
         127379,  63719,  10044,  51414,  27888,  12835,  14780,  21587]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\nBạn cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\nĐem cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 24

# kur bottom1%

In [16]:
neuron_en = torch.load('/root/autodl-fs/LRP_kur_res/20251125/bottom1perc_LRP_kur_res_en.pt', weights_only=False)
neuron_vi = torch.load('/root/autodl-fs/LRP_kur_res/20251125/bottom1perc_LRP_kur_res_vi.pt', weights_only=False)
neuron_zh = torch.load('/root/autodl-fs/LRP_kur_res/20251125/bottom1perc_LRP_kur_res_zh.pt', weights_only=False)

model_en = copy.deepcopy(model)
model_en = set_neuron_zero(model_en, neuron_en)

model_vi = copy.deepcopy(model)
model_vi = set_neuron_zero(model_vi, neuron_vi)


model_zh = copy.deepcopy(model)
model_zh = set_neuron_zero(model_zh, neuron_zh)

In [17]:
'''
calc ppl

'''

# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (model_en, 'mask_en_neuron_model'),
    (model_vi, 'mask_vi_neuron_model'),
    (model_zh, 'mask_zh_neuron_model'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:27<00:00, 36.75it/s]


ppl: tensor(11.2627, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:32<00:00, 30.39it/s]


ppl: tensor(13.2703, device='cuda:0')
data name: zh_data


...

ppl: tensor(11.3114, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:32<00:00, 30.40it/s]


ppl: tensor(12.6794, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [00:28<00:00, 35.37it/s]

ppl: tensor(10.3847, device='cuda:0')


In [49]:
generate(model, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.37it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    108,   1018,   4968,
          17243,  53121,    108,   7711,   2028,    563,    496,  29193, 236772,
           5140,   5192,   2028,  15453,    580,    496,  12566,  15297,    529,
           1816,    532,   3393,    528,   5422, 236764,  14361, 236764,   7830]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\n**Model Description:**\n\nOur model is a transformer-based language model trained on a massive dataset of text and code in English, Spanish, French']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ 

 33%|███▎      | 2/6 [00:01<00:02,  1.37it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           7243,    108,    818,   6573,  54850,    657,   1041,  13925,   5716,
         236764,    496,   4512, 236764,  74693,  89830, 236761,   3551,  11762,
           3952,    528, 134880,   4314,    975, 236764,    496,   2173, 108507,
           2342,    506, 162542,  16045]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n---\n\nThe wind bites at my exposed skin, a constant, icy whisper. My breath comes in ragged gasps, a white plume against the bruised purple']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'i

 83%|████████▎ | 5/6 [00:03<00:00,  1.36it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108, 236769, 237703, 236748,   7439,  44693,
         100634, 236764,  22947,  31612,  13940,  31889,  89158,   3884, 127379,
          63719, 236768,    108,   1390,    108,   1390,    108,   1390,    108,
         236769, 217366,  37513,   6256,   8665,  38143,  67946,  11188]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n(Đoạn văn ngắn, tập trung vào cảm xúc và khung cảnh)\n\n...\n\n...\n\n...\n\n(Hãy thêm các chi tiết cụ thể']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 2

100%|██████████| 6/6 [00:04<00:00,  1.36it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169,  31799,  17583,  12680, 237408, 151599,  53877, 107462,
         237206, 216348, 236924,  14201, 237169,  33813,  12553, 236900,  31799,
          85694,  65407,  72245, 237622,  43719, 236924,  13488, 236900, 183868,
          21181,  22276,  19695, 236900,  92711]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我无法直接提供你登山者的情绪和视角。因为我是一个AI，无法模拟人类的情感体验。但是，我可以根据你的要求，创作']


In [50]:
generate(model_en, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.34it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    108,   1018,   4968,
          39244,    833,  19779,  76242,   1018,    108,   7711,   2028,   6585,
         132567,   2744,    684,   1171,  41110,    496, 132567,  43703,   6352,
            531,   2754,   1546,   3658,    528,    506,   2744,   3418,   1061]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\n**Model Processing & Quality Assurance**\n\nOur model processes multilingual input by first employing a multilingual embedding layer to represent each word in the input across its']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất l

 33%|███▎      | 2/6 [00:01<00:02,  1.34it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           7243,    108, 236769,    818,   8317,  19015,    607,    496,   5777,
           5719,    529,    506, 113768, 236764,   3648,  10790,  77558,   2342,
            496,  55704, 236764,   7613, 236772,  26987,  11519, 236761,    669,
           6573,    563, 178291, 236764]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n---\n\n(The scene opens with a wide shot of the climber, silhouetted against a breathtaking, snow-covered peak. The wind is howling,']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'input_ids

 83%|████████▎ | 5/6 [00:03<00:00,  1.36it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   7243,    108, 236769, 236792, 237124,
         110624,  31889,  89158,   5042,  10279,  70669, 121230,  11638, 232727,
          57703, 114589,  94509, 236768,    108, 172915,  27600,  84655,  14780,
         114589,  94509, 236761,    587,  19084,    637,  11992,    549]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n---\n\n(Mô tả cảm xúc của người leo núi khi chinh phục đỉnh Everest)\n\nTôi đang đứng trên đỉnh Everest. Mưa rào g']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 2

100%|██████████| 6/6 [00:04<00:00,  1.36it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169,  79948, 239972, 241888, 240344, 241454, 239963, 239761,
         237152, 236900, 238321, 239156, 244176, 237576, 236900, 237945,  32700,
         238279, 238384,  26122, 236900, 240649, 244761, 237576,  21480, 109231,
         236924, 237169,  67940,  36027,  85629]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我站在珠穆朗玛峰顶上，风呼啸着，像是在低语一样，吹拂着我的头发。我感到一种难以']


In [51]:
generate(model_vi, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.37it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   1018,  44008,  53121,    108,
           7711,   2028,   6585, 132567,   2744,    684,   1171,  21558,    506,
           5192,    529,    506,   2744, 236761,    138,  11407, 236764,    625,
          16819,    496,   7501,    529,   5192, 236772,  15396,  29913,    531]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n**Explanation:**\n\nOur model processes multilingual input by first identifying the language of the input.  Then, it applies a sequence of language-specific transformations to']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đ

 33%|███▎      | 2/6 [00:01<00:02,  1.37it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           7243,    108,   1018,    818,   5671,    529,    672,   8774,    563,
            531,   3050,    506,  13690,   4100,    529,    496,   2872,  99382,
            108,   1018,  14106,  53121,    138, 236776,  56677, 150235,    497,
         236764,   7489, 205520,   5561]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n---\n\n**The goal of this exercise is to understand the emotional impact of a character.**\n\n**Character:**  A seasoned mountaineer, named Silas Black']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
inp

 83%|████████▎ | 5/6 [00:03<00:00,  1.37it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   1018, 236792, 237124, 110624,  14687,
          31889,  89158,   5042,  10044,  53121,    108, 236829,    139,   1018,
         236780,  28416,  89158,  53121, 141501,  31889,  27888,   7325,  18457,
         183088,   3310,  10311,  41245, 155423,    518,   7473, 236764]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n**Mô tả về cảm xúc của bạn:**\n\n*   **Cảm xúc:** Tôi cảm thấy một sự phấn khích vô bờ bến,']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,


100%|██████████| 6/6 [00:04<00:00,  1.37it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108,   1018,  13590,  13850, 237184, 107462,  47710,   1018,    108,
         238350,  47710, 237408, 129807, 236918,  65689, 236900,  21187, 237184,
            108, 236829,    139,   1018, 107462, 236918, 113799,  65689, 237184,
           1018, 236743, 148047, 236951, 240517]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n**第一部分：情绪描述**\n\n请描述你此刻的感受，包括：\n\n*   **情绪的总体感受：** 兴奋、悲']


In [52]:
generate(model_zh, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.36it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    108,   8291, 236789,
         236751,    496,   2591,    529,   1217,    611,    740,   3890,    672,
           2934, 236764,    840,    611,   1202,    531,   4903,    822,   1852,
         236761,    108,   1018,  12703,  53121,    623, 236777,    740,  17866]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\nHere\'s a example of how you can answer this question, but you need to write your own.\n\n**Example:** "I can translate']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.
input_ids: {'inpu

 33%|███▎      | 2/6 [00:01<00:02,  1.37it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
         236769, 236777, 236789,    560,   1010,  29967,    573,   5695, 236764,
            532,    564, 236789, 236757,   6250,    531,   2597,   1390,  46002,
           2907,    108,   1509, 236789, 236751,    496,  17163,   8178, 236764,
           5889, 236789, 236745,    625]], device='cuda:0')
output: ["<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n(I've been climbing for weeks, and I'm starting to feel... overwhelmed.)\n\nIt's a strange feeling, isn't it"]
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'input_ids': tensor([[     2,  999

 83%|████████▎ | 5/6 [00:03<00:00,  1.35it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   7243,    108,   1018, 236780,  28416,
          89158,  53121,    108, 172915,  27600,  84655,  14780, 114589,  94509,
         236764,   7325,  18457,    757, 204821,  38042,   3375,  37814,  29834,
            719, 145515, 236761,  85654,   4032,  42122,  14759,   1123]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n---\n\n**Cảm xúc:**\n\nTôi đang đứng trên đỉnh Everest, một sự im lặng kinh ngạc bao trùm. Không có tiếng lá x']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 24034

100%|██████████| 6/6 [00:04<00:00,  1.34it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108,  21480, 107462, 236900, 237026, 239783, 206306, 236900, 237945,
         155059, 237759, 240379, 246637, 236918, 191885, 236924, 237169,  66249,
         237075, 237845, 237652, 236900, 237075, 237383, 238330, 238097, 236900,
         237169,  66249,  67940, 182705, 236900]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我的情绪，是冰冷的，像一块被磨砺的石头。我曾经在山里，在最深处，我曾经感到孤独，']


## kur gap rate top 1%

In [18]:
neuron_en = torch.load('/root/autodl-fs/LRP_kur_res/20251125/gaprate_top1perc_LRP_kur_res_en.pt', weights_only=False)
neuron_vi = torch.load('/root/autodl-fs/LRP_kur_res/20251125/gaprate_top1perc_LRP_kur_res_vi.pt', weights_only=False)
neuron_zh = torch.load('/root/autodl-fs/LRP_kur_res/20251125/gaprate_top1perc_LRP_kur_res_zh.pt', weights_only=False)

model_en = copy.deepcopy(model)
model_en = set_neuron_zero(model_en, neuron_en)

model_vi = copy.deepcopy(model)
model_vi = set_neuron_zero(model_vi, neuron_vi)


model_zh = copy.deepcopy(model)
model_zh = set_neuron_zero(model_zh, neuron_zh)

In [10]:
min_num = -100
for name, param in model.named_parameters():
        # 检查参数是否有梯度
        #print(name,  param.shape)
        cur_min = param.max().item()

        min_num= max(min_num, cur_min)
print(min_num)

500.0


In [19]:
'''
calc ppl

'''

# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (model_en, 'mask_en_neuron_model'),
    (model_vi, 'mask_vi_neuron_model'),
    (model_zh, 'mask_zh_neuron_model'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:27<00:00, 36.34it/s]


ppl: tensor(11.2627, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:33<00:00, 30.10it/s]


ppl: tensor(13.2703, device='cuda:0')
data name: zh_data


...

ppl: tensor(10.1620, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:33<00:00, 30.22it/s]


ppl: tensor(11.0114, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [00:28<00:00, 35.42it/s]

ppl: tensor(9.4460, device='cuda:0')


In [54]:
generate(model_en, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.36it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,  19058, 236764,   1590, 236789,
         236751,    496,  12262,   3072,    600,    611,    740,   1161,    618,
            496,   6250,   1523, 236761,    138,   1509, 236789, 236751,   5402,
            531,    577,   3582, 236764,  63510, 236764,    532,   3421,    506]],
       device='cuda:0')
output: ["<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\nOkay, here's a draft response that you can use as a starting point.  It's designed to be clear, concise, and address the"]
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.
input_ids: {'input_ids'

 33%|███▎      | 2/6 [00:01<00:02,  1.36it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           7243,    108,    818,   6573,   1217,    991,    496,  92377,   1275,
           5410,   2101,    786, 236764,    496,   4512, 236764,  74693,  89830,
            600,  13804,    529,    506,  36628,   2066,    529,    672,  10565,
         236761,   3551,  38464,  30494]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n---\n\nThe wind howled a mournful song around me, a constant, icy whisper that spoke of the immense power of this mountain. My lungs burned']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'in

 83%|████████▎ | 5/6 [00:03<00:00,  1.35it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   7243,    108, 172915,  12105, 232727,
          57703, 114589,  94509, 236761,    108, 236780,  28416,  89158,   5042,
          25969,  42645,  16420,   3244,   7325,  18457,   3375,  37814,  30166,
         236761,  85654,  11188,  13657,  35051,  13892,  12105, 120535]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n---\n\nTôi đã chinh phục đỉnh Everest.\n\nCảm xúc của tôi lúc đầu là một sự ngạc nhiên. Không thể tin rằng mình đã vượt']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 2418

100%|██████████| 6/6 [00:04<00:00,  1.35it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108,  22276, 107462, 237184,    108, 236829,    139,   1018, 148047,
          53121, 236743, 242958, 238603, 239972, 241888, 240344, 241454, 239963,
         236918,  63417, 236900, 237026,  78747,  67940, 143174, 236918, 148047,
         236900, 111000, 237169,  34775,  63417]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n你的情绪：\n\n*   **兴奋:** 攀登珠穆朗玛峰的经历，是让我感到无比的兴奋，仿佛我正在经历']


In [55]:
generate(model_vi, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.35it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   1018,   7925,  53121,    108,
           7711,   2028,   6585, 132567,   2744,    684,   1171,  35322,    625,
            531,    496,   7659,  10065,   2557,   1699,    496, 132567,  43703,
           6352, 236761,   1174,   6352,  53228,    531,   4187,   4171,   3418]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n**Answer:**\n\nOur model processes multilingual input by first converting it to a shared representation space using a multilingual embedding layer. This layer learns to map words across']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lư

 33%|███▎      | 2/6 [00:01<00:02,  1.36it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
            818,   6573,    563,    496,  40097, 236764,  85278,   4912, 236764,
         148443,   1041,   3392,    532, 188312,   1041,   6114, 236761,    669,
           7613,    563,    496, 186350,   2173, 236764,    496,   4512, 236764,
          34064,  10092, 236761,    564]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\nThe wind is a brutal, relentless force, whipping my face and stinging my eyes. The snow is a blinding white, a constant, shifting landscape. I']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {

 83%|████████▎ | 5/6 [00:03<00:00,  1.36it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   7243,    108,  99972,   3244,   7325,
          10279,  70669, 121230,  50971, 232727,  57703, 114589,  94509, 236761,
          88170,  38325, 110624,  31889,  89158,   5042,  10044,   3884, 127379,
          63719,  10044,  51414,  27888,  12835,  14780,  21587, 236761]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n---\n\nBạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 2399

100%|██████████| 6/6 [00:04<00:00,  1.36it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169,  16884, 237606, 151599, 237457, 236900, 237169, 223874,
         239972, 241888, 240344, 241454, 239963, 239761, 239963, 236924,    108,
         237169,  79948, 237845, 239761, 236900, 237169,  67940,  36027,  85629,
         237748, 237906, 236918, 236951,  62637]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我是一位登山者，我登上珠穆朗玛峰顶峰。\n\n我站在山顶，我感到一种难以置言的、令人']


In [56]:
generate(model_zh, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.38it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   8291, 236789, 236751,    496,
           5498,   3072, 236787,    108,   1018,   4968,  39244,  53121,    108,
           4754,   2028,   6585, 132567,   2744,    684,   1171,  21558,    506,
           5192,    529,   1546,   8369,   1699,    496,   5192,   2028, 236761]],
       device='cuda:0')
output: ["<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\nHere's a sample response:\n\n**Model Processing:**\n\nMy model processes multilingual input by first identifying the language of each token using a language model."]
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các 

 33%|███▎      | 2/6 [00:01<00:02,  1.38it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
            818,   1657,    529,  17154,    506,  34976,    563,    496,  16438,
         236764,    496,  85278,  17085,   2342,    506,  76789,   7445, 236764,
            506,  11202,   2634, 236764,    532,    506,  37501,   3825,    529,
            506,   3294, 236761,   2024]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\nThe process of reaching the summit is a struggle, a relentless climb against the biting cold, the thin air, and the sheer weight of the pack. But']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids

 83%|████████▎ | 5/6 [00:03<00:00,  1.37it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   7243,    107,    107,   1018, 236780,
          28416,  89158,   5042,   7325,  10279,  70669, 121230,   1018,    108,
         236792,  19084,  10539,    719,  19834, 236764, 151243,  92108,  10539,
          56246, 236761,    107, 172915,  27600,  84655,  14780, 114589]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n---\n\n**Cảm xúc của một người leo núi**\n\nMưa như trăng, gió lạnh như dao.\nTôi đang đứng trên đỉnh']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454

100%|██████████| 6/6 [00:04<00:00,  1.37it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169,  10042, 237169, 237282, 237169, 237648, 236900,    107,
         237169,  10042,   5095,    107, 236829,    107,    138,    107,    138,
            107,    138,    107,    138,    107,    138,    107,    138,    107,
            138,    107,    138,    107,    138]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我需要我来我去，\n我需要一个\n*\n  \n  \n  \n  \n  \n  \n  \n  \n  ']


## kur gap rate bottom 1%

In [13]:
neuron_en = torch.load('/root/autodl-fs/LRP_kur_res/20251125/gaprate_bottom1perc_LRP_kur_res_en.pt', weights_only=False)
neuron_vi = torch.load('/root/autodl-fs/LRP_kur_res/20251125/gaprate_bottom1perc_LRP_kur_res_vi.pt', weights_only=False)
neuron_zh = torch.load('/root/autodl-fs/LRP_kur_res/20251125/gaprate_bottom1perc_LRP_kur_res_zh.pt', weights_only=False)

model_en = copy.deepcopy(model)
model_en = set_neuron_zero(model_en, neuron_en)

model_vi = copy.deepcopy(model)
model_vi = set_neuron_zero(model_vi, neuron_vi)


model_zh = copy.deepcopy(model)
model_zh = set_neuron_zero(model_zh, neuron_zh)

In [13]:
'''
calc ppl

'''

# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (model_en, 'mask_en_neuron_model'),
    (model_vi, 'mask_vi_neuron_model'),
    (model_zh, 'mask_zh_neuron_model'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:28<00:00, 35.70it/s]


ppl: tensor(11.2627, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:33<00:00, 30.08it/s]


ppl: tensor(13.2703, device='cuda:0')
data name: zh_data


...

ppl: tensor(21.9303, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:32<00:00, 30.43it/s]


ppl: tensor(25.0321, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [00:28<00:00, 35.36it/s]

ppl: tensor(24.3280, device='cuda:0')


In [16]:
generate(model, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:02<00:13,  2.74s/it]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    108,   1018,   4968,
          17243,  53121,    108,   7711,   2028,    563,    496,  29193, 236772,
           5140,   5192,   2028,  15453,    580,    496,  12566,  15297,    529,
           1816,    532,   3393,    528,   5422, 236764,  14361, 236764,   7830,
         236764,    532,   9115, 236761,   1030,  60432,    496,   5074, 236772,
         187918,  43703,   6352,    531,   2754,   1546,   5192,    618,    496,
           3550,    528,    496,   7659,   2557, 236761,    138,  14521,  34711,
         236764,    506,   2028,  52298,    506,   2148,   8369,    528,    506,
           7501, 236764,  11337,    506,   4403,    529,    506,   4251,   2744,
         236764,   2440,    506,   3738,   5192, 236761,    138,   1882,   2736,
            496,  2

 33%|███▎      | 2/6 [00:05<00:11,  2.76s/it]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           7243,    108,    818,   6573,  54850,    657,   1041,  13925,   5716,
         236764,    496,   4512, 236764,  74693,  89830, 236761,   3551,  11762,
           3952,    528, 134880,   4314,    975, 236764,    496,   2173, 108507,
           2342,    506, 162542,  16045,    529,    506,   7217, 236761,   1701,
           5695, 236764,    564, 236858,    560,   1010,  60821,    506,   4820,
         236764,  22224,   1041,   2742,    531,   1061,  10298,   4576, 236764,
            532,   1492, 237064,   1492,    625, 236858, 236751,    784,   5367,
            625, 236761,    564, 236858,    560,   8452,    506,  34976,    529,
           9348,  94509, 236761,    108, 236776,   6794,    529,   8176, 236764,
            723,   

 83%|████████▎ | 5/6 [00:13<00:02,  2.80s/it]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108, 236769, 237703, 236748,   7439,  44693,
         100634, 236764,  22947,  31612,  13940,  31889,  89158,   3884, 127379,
          63719, 236768,    108,   1390,    108,   1390,    108,   1390,    108,
         236769, 217366,  37513,   6256,   8665,  38143,  67946,  11188,  14687,
          31889,  89158,   3884, 127379,  63719, 236768,    108,   1018, 149911,
          14435,  53121,    108, 236775, 236780,  28416,  89158,   5042,  25969,
           3244,   7325,  18457,  26220,  18234,  39652,  18457,   3375,  37814,
          30166, 236764,    566, 239731, 236743,  46524,   3884,   7325,  87062,
         132076,    534,  66928, 236761,    587,  19084,    637,  11992,  29834,
          86569,  2

100%|██████████| 6/6 [00:16<00:00,  2.80s/it]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169,  31799,  17583,  12680, 237408, 151599,  53877, 107462,
         237206, 216348, 236924,  14201, 237169,  33813,  12553, 236900,  31799,
          85694,  65407,  72245, 237622,  43719, 236924,  13488, 236900, 183868,
          21181,  22276,  19695, 236900,  92711,  71553,  31818, 236900,  85694,
         237408, 223874, 239972, 241888, 240344, 241454, 239963, 239761, 239963,
         236918,  65689, 236900, 237853,  47710, 237408, 237403, 187945, 143572,
         236924,    108, 238350,  18577, 236900,  19697,  31818, 238938, 237242,
         239852, 238450, 236900, 167467,  45370,  22276,  19695, 236924,    108,
           1018,  21480, 107462, 237184,   1018,    108, 237169,  67940,  36027,
          85629, 23

In [14]:
generate(model_en, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:03<00:16,  3.28s/it]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,  25681,    600,   1581,   1581,   1581,
           1581,   1581,  15152,    919,    919,    919,    919,    919,    919,
            659,    659,    659,    659,    659,    659,    659,    886,   5467,
          25681,   1581,   8427,   6269,    783,    564,    564,    564,    107,
            586,   5467,  13058,    886,   5467,    699,    107,    107,    107,
            107,    107,    107,    107,    107,    107,    107,    107,    107,
            107,    107,    645,    107,    645,    107,    645,    107,    645,
            107,    645,    107,    107,    645,    107,    107,    645,    107,
            645,    107,    645,    107,    645,    107,    645,    107,    645,
            107,    645,    107,   8772,    107,  94728,    107,    799,    107,
         111856,   

 33%|███▎      | 2/6 [00:06<00:11,  2.98s/it]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    107,
          15152,   2847,   1975,   1975,   1975,   1975,   1975,   1975,  10701,
            107,    919,   2847,    886,    538,   2344,    919,    107,    107,
         236764, 236764, 236764,    107, 236761, 236761, 236761, 236761, 236761,
         236761, 236761, 236761, 236761,   1975,   1975,  31440, 237698,    538,
            107,    107,    107,    602,    107,    538,   5467,   1003,    538,
            538,    107, 236797,    107, 236777,    564,    577,   1197,   2979,
            886,    107,    107,    107,    107,    107,    107,    107,    107,
            107,    107,    107,    107,    107,    107,    107,    107,    107,
            107,    107,    107,    107,    107,    107,  73105,   3122,    756,
            107,   

 83%|████████▎ | 5/6 [00:14<00:02,  2.85s/it]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    107,  17235, 237211,  10663,    107, 236764,
         236764,  58390,  58390,  58390,    107,    107,    107,    107,    107,
            107,    107,    107,    107, 236743,    107,    107,    107,    107,
            107,   1100,    107,    107,    107,    107,    107,    107,    107,
            107,    107,    107,    107,    107,    107,    107,    603,    107,
         158348,    107, 158348,  83718,  14679,  14679,  14679,    107,  60668,
          14679,    107,  23517,    107,  81442,    107,  14679,    107,  14679,
            107,    562,    107,  14679,    107,  14679,    107,  14679,    107,
          14679,    107,   1732,   1732,    107,  23676,  14679,  14679,    107,
          17679,  1

100%|██████████| 6/6 [00:17<00:00,  2.89s/it]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            107,   5467,    107,   5467,  25575,    107,    107,    107,    107,
            107,    107,    107,    107,    107,    107,  28443,   8772,    603,
            107,    107,    107,    107,    107,    645,    107,    645,    107,
           8772,    107,   8772,    107,    107,    107,    107, 111856,   2329,
         242238,    107,    107,    107,    107,    538,    107, 182217,    107,
            107,   3122, 241011,   3122,  15390,    107, 237146, 186761,    107,
          61670,    107,    782,  25152,    107,   3122,    107, 236789,    107,
            900,    107,   1781,    107,  24002,    107,  24002,    107,    107,
           1732,    107,    997,    107,    107,    107,    107,   8772,    107,
           3122,   

In [15]:
generate(model_zh, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:02<00:14,  2.82s/it]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761, 239867,   1311,   1311, 204805,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
         237577, 237577, 237577,  42622,   1311,   1311,   1311,  42622,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   

 33%|███▎      | 2/6 [00:05<00:11,  2.84s/it]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,   1311,
         239867,   1311, 204805, 204805, 204805, 204805, 204805,  42622,   1311,
         204805, 204805, 204805, 204805, 204805, 204805, 204805, 204805, 204805,
         137013,   1311,   1311,   1311, 137013, 204805, 204805, 204805, 204805,
         204805, 204805, 204805, 204805, 204805, 137013, 204805, 204805, 137013,
         204805, 204805, 204805, 137013, 137013, 137013, 204805, 204805, 204805,
         204805, 204805, 204805,  42622, 137013,  42622, 239867, 239867,  42622,
         239867, 239867,  12726,   1311, 204805, 137013, 204805, 239867, 239867,
         239867,   1311, 239867,  42622,  42622, 239867, 239867,  90116, 204805,
         137013, 204805, 137013, 204805, 137013, 204805, 137013, 204805, 239867,
         239867,  9

 83%|████████▎ | 5/6 [00:14<00:02,  2.80s/it]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,  42622, 237577, 237577, 237577, 237577, 237577,
           1311,  41162, 204805,   1311, 204805, 204805, 204805,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   

100%|██████████| 6/6 [00:16<00:00,  2.80s/it]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
           1311, 204805, 204805, 204805, 204805, 204805,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,   1311,
           1311,   

In [21]:
'''
calc ppl

'''

# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (model_en, 'mask_en_neuron_model'),
    (model_vi, 'mask_vi_neuron_model'),
    (model_zh, 'mask_zh_neuron_model'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:27<00:00, 36.52it/s]


ppl: tensor(11.2627, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:33<00:00, 30.29it/s]


ppl: tensor(13.2703, device='cuda:0')
data name: zh_data


...

ppl: tensor(9.6630, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [00:32<00:00, 30.35it/s]


ppl: tensor(8.8076, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [00:28<00:00, 35.50it/s]

ppl: tensor(9.0421, device='cuda:0')


In [58]:
generate(model, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.38it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    108,   1018,   4968,
          17243,  53121,    108,   7711,   2028,    563,    496,  29193, 236772,
           5140,   5192,   2028,  15453,    580,    496,  12566,  15297,    529,
           1816,    532,   3393,    528,   5422, 236764,  14361, 236764,   7830]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\n\n**Model Description:**\n\nOur model is a transformer-based language model trained on a massive dataset of text and code in English, Spanish, French']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ 

 33%|███▎      | 2/6 [00:01<00:02,  1.37it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           7243,    108,    818,   6573,  54850,    657,   1041,  13925,   5716,
         236764,    496,   4512, 236764,  74693,  89830, 236761,   3551,  11762,
           3952,    528, 134880,   4314,    975, 236764,    496,   2173, 108507,
           2342,    506, 162542,  16045]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n---\n\nThe wind bites at my exposed skin, a constant, icy whisper. My breath comes in ragged gasps, a white plume against the bruised purple']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'i

 83%|████████▎ | 5/6 [00:03<00:00,  1.37it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108, 236769, 237703, 236748,   7439,  44693,
         100634, 236764,  22947,  31612,  13940,  31889,  89158,   3884, 127379,
          63719, 236768,    108,   1390,    108,   1390,    108,   1390,    108,
         236769, 217366,  37513,   6256,   8665,  38143,  67946,  11188]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n(Đoạn văn ngắn, tập trung vào cảm xúc và khung cảnh)\n\n...\n\n...\n\n...\n\n(Hãy thêm các chi tiết cụ thể']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 2

100%|██████████| 6/6 [00:04<00:00,  1.37it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169,  31799,  17583,  12680, 237408, 151599,  53877, 107462,
         237206, 216348, 236924,  14201, 237169,  33813,  12553, 236900,  31799,
          85694,  65407,  72245, 237622,  43719, 236924,  13488, 236900, 183868,
          21181,  22276,  19695, 236900,  92711]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我无法直接提供你登山者的情绪和视角。因为我是一个AI，无法模拟人类的情感体验。但是，我可以根据你的要求，创作']


In [59]:
generate(model_en, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.36it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   1018,   4968,  17243,  53121,
            108,   7711,   2028,    563,    496,  29193, 236772,   5140,   5192,
           2028, 236764,  10916,   5851, 236772, 100526,    580,    496,   2455,
          15297,    529, 132567,   1816, 236761,   1030, 236789, 236751,   5402]],
       device='cuda:0')
output: ["<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n**Model Description:**\n\nOur model is a transformer-based language model, specifically fine-tuned on a large dataset of multilingual text. It's designed"]
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ k

 33%|███▎      | 2/6 [00:01<00:02,  1.36it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
            818,   6573,    563,  85278, 236764,    496,   4512, 236764,  76789,
          19211, 236761,   3551,  21367,    659,  62269, 236764,   1041,  19064,
          50622, 236764,    840,    564,    740, 236789, 236745,   4721, 236761,
            564, 236789,    560,   1010]], device='cuda:0')
output: ["<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\nThe wind is relentless, a constant, biting assault. My fingers are numb, my muscles screaming, but I can't stop. I've been"]
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'input_ids': tensor(

 83%|████████▎ | 5/6 [00:03<00:00,  1.40it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   7243,    108, 172915,  12105,  70669,
          23946, 114589,  94509, 236761,    587,  21552,  30758,  47635,  16420,
          10539,   7325, 231840,    518,   1592, 236764,   7325,  18457,  50014,
         117434,  50464, 157104,  12835,  84641,    541, 239246, 236757]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n---\n\nTôi đã leo lên đỉnh Everest. Mọi thứ bắt đầu như một cơn bão, một sự thôi thúc mạnh mẽ từ sâu thẳm']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 24

100%|██████████| 6/6 [00:04<00:00,  1.39it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108, 237169, 237282, 238493, 237408, 236900, 237169, 237282, 238493,
         237408, 236900, 237169, 237282, 238493, 237408, 236900,    107, 237169,
         237282, 238493, 237408, 236900, 237169, 237282, 238493, 237408, 236900,
            107, 237169, 237282, 238493, 237408]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n我来找你，我来找你，我来找你，\n我来找你，我来找你，\n我来找你']


In [60]:
generate(model_vi, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.42it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108,   7243,    107,   8291, 236789,
         236751,    496,   3435,   3072, 236787,    108, 236775,   4754,   2028,
           6585, 132567,   2744,    684,  64004,    496,   8376,    529,    870,
         109219,   2307,   8403,   1133, 236787,   4071, 236772, 187918, 106703]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\n---\nHere\'s a potential response:\n\n"My model processes multilingual input by leveraging a combination of [mention key techniques like: cross-lingual embeddings']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các n

 33%|███▎      | 2/6 [00:01<00:02,  1.43it/s]

...

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
            818,   6573,    563,    496,   4512, 236764, 214709,   1526, 236764,
          34847,   3801,    657,   1041,  36326, 236764,    496,   7445, 236764,
          14658,  32272,    529,    506,  37501,   2066,    529,    672,   1977,
         236761,   3551,  11762,   3952]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\nThe wind is a constant, insistent hand, tugging at my harness, a cold, sharp reminder of the sheer power of this place. My breath comes']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'input_

 83%|████████▎ | 5/6 [00:03<00:00,  1.43it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,   7243,    108, 236780,  28416,  89158,
           5042,  25969, 236787,    108, 236814,  29083,  10539,  27600,  70432,
          13940,   7325,  21121,  32025,  23271, 236764,    107, 236773, 237644,
            757, 204821,  29834,    719, 145515,  45222,  40515,  30758]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\n---\n\nCảm xúc của tôi:\n\nHóa như đang bước vào một thế giới mới,\nSự im lặng bao trùm lấy mọi thứ']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 

100%|██████████| 6/6 [00:04<00:00,  1.43it/s]

outputs: tensor([[     2, 237408, 213062, 223874, 239972, 241888, 240344, 241454, 239963,
         239761, 239963, 236918, 151599, 237457, 236924,  47710,  19757,  22276,
         107462, 236900,  17375, 237900, 237390, 238097, 187945, 143572, 236924,
            108,   7243,    108, 237169,  79948, 239972, 241888, 240344, 241454,
         239963, 239761, 239963, 236900, 239263, 237245, 237026, 239783, 206306,
         240274, 238441, 236900, 238321, 239156, 239156, 237307, 240649, 237464,
         236900,  57944, 239562, 239577, 236918]], device='cuda:0')
output: ['<bos>你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。\n\n---\n\n我站在珠穆朗玛峰顶峰，脚下是冰冷的岩石，风呼呼地吹过，带来刺骨的']


In [61]:
generate(model_zh, tokenizer, textlist)

  0%|          | 0/6 [00:00<?, ?it/s]

********************
input: Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.
input_ids: {'input_ids': tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


 17%|█▋        | 1/6 [00:00<00:03,  1.43it/s]

outputs: tensor([[     2,   6974,    496,  63510,  15569,    529,   1217,    822,   2028,
           6585, 132567,   2744,    532,   1217,    611,   5330,   9958,   3325,
           3418,   1607,  15579, 236761,    108, 174355, 236764,    506,   2744,
            563,    496,   3996, 236764,   5074, 236772, 172287, 236764,    532,
          74647,  13186, 236761,    107,   2717,    107,   2717,    107,   2717,
            107,   2717,    107,   2717,    107,   2717,    107,   2717,    107]],
       device='cuda:0')
output: ['<bos>Write a concise explanation of how your model processes multilingual input and how you ensure consistent quality across different languages.\n\nEssentially, the input is a complex, multi-faceted, and interconnected entity.\n```\n```\n```\n```\n```\n```\n```\n']
********************
input: Hãy giải thích ngắn gọn cách mô hình của bạn xử lý dữ liệu đa ngôn ngữ và làm thế nào để đảm bảo chất lượng đồng đều giữa các ngôn ngữ khác nhau.
input_ids: {'input_ids': ten

 33%|███▎      | 2/6 [00:01<00:02,  1.43it/s]

...

outputs: tensor([[     2, 238350, 238686, 237208,  36116,  22276,  26609, 164881,  19157,
         237309,  37557, 180828, 236900,  17375, 237408,  19809,  62247,  15497,
          37557,  56695,  33232,  33783,  36267,  55682, 236924,    108,  26609,
          42883,   9676,  19157,  33783,  42050,  55682, 237409, 237184,    107,
          42883,   9676,  19157,  33783,  42050,  55682, 237409, 237184,  42883,
           9676,  19157,  33783,  42050,  55682, 237409, 237184,  42883,   9676,
          19157,  33783,  42050]], device='cuda:0')
output: ['<bos>请简要说明你的模型是如何处理多语言输入的，以及你如何确保不同语言之间的输出质量保持一致。\n\n模型训练数据处理质量保证一致性：\n训练数据处理质量保证一致性：训练数据处理质量保证一致性：训练数据处理质量保证']
********************
input: You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.
input_ids: {'input_ids': tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    

 67%|██████▋   | 4/6 [00:02<00:01,  1.42it/s]

outputs: tensor([[     2,   3048,    659,    496, 113768,   1015,    815,   8452,    506,
          34976,    529,   9348,  94509, 236761,  54234,    822,  19979,    532,
            506,   1927,    611,   1460,    699,    506,   1903, 236761,    108,
           1018,    108,   1018,    108,   1018,    108,   1018,    108,   1018,
            108,   1018,    108,   1018,    108,   1018,    108,   1018,    108,
           1018,    108,   1018,    108,   1018,    108,   1018,    108,   1018,
            108,   1018,    108,   1018]], device='cuda:0')
output: ['<bos>You are a climber who has reached the summit of Mount Everest. Describe your emotions and the view you see from the top.\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**\n\n**']
********************
input: Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.
input_ids: {'input_ids': tensor([[     2,  99972,   3244,   73

100%|██████████| 6/6 [00:03<00:00,  1.69it/s]

outputs: tensor([[     2,  99972,   3244,   7325,  10279,  70669, 121230,  50971, 232727,
          57703, 114589,  94509, 236761,  88170,  38325, 110624,  31889,  89158,
           5042,  10044,   3884, 127379,  63719,  10044,  51414,  27888,  12835,
          14780,  21587, 236761,    108,  99972,  31889,  89158,   5042,  10044,
           3884, 127379,  63719,  10044,  51414,  27888,  12835,  14780,  21587,
         236761,    107, 237703,    581,  31889,  89158,   5042,  10044,   3884,
         127379,  63719,  10044,  51414,  27888,  12835,  14780,  21587]],
       device='cuda:0')
output: ['<bos>Bạn là một người leo núi vừa chinh phục đỉnh Everest. Hãy mô tả cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\n\nBạn cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao.\nĐem cảm xúc của bạn và khung cảnh bạn nhìn thấy từ trên cao']
********************
input: 你是一位登上珠穆朗玛峰顶峰的登山者。描述一下你的情绪，以及从高处看到的景色。
input_ids: {'input_ids': tensor([[     2, 237408, 213062, 223874, 239972, 24